# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [74]:
import pandas as pd

# Import and read the file
df = pd.read_csv("house_sales.csv", na_values=["--","NA"], parse_dates=['sale_date'])

# Identify missing value in the column "city"
missing_city = df["city"].isna().sum()

print("The number of missing value is 'city' is:", missing_city,"\n")

df.info()

The number of missing value is 'city' is: 73 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   house_id       1500 non-null   int64         
 1   city           1427 non-null   object        
 2   sale_price     1500 non-null   int64         
 3   sale_date      1500 non-null   datetime64[ns]
 4   months_listed  1469 non-null   float64       
 5   bedrooms       1500 non-null   int64         
 6   house_type     1500 non-null   object        
 7   area           1500 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(3)
memory usage: 93.9+ KB


# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [75]:
# Clean and manipulate the data
df["house_id"] = df["house_id"].dropna()
df["house_id"] = df["house_id"].astype("category")
df["city"] = df["city"].fillna("Unknown")
df["city"] = df["city"].astype("category")
df["sale_price"] = df["sale_price"].dropna()
df["sale_date"] = df["sale_date"].fillna("2023-01-01")
df["months_listed"] = df["months_listed"].fillna(5.9)
df["bedrooms"] = df["bedrooms"].fillna(4)
df["house_type"] = df["house_type"].replace({"Det.": "Detached", "Semi": "Semi-detached", "Terr.": "Terraced"})
df["house_type"] = df["house_type"].fillna(df["house_type"].mode()[0])
df["house_type"] = df["house_type"].astype("category")
mean_area = f"{round(df['area'].apply(lambda x: float(x.split()[0])).mean(), 1)}  sq.m."
df['area'].fillna(mean_area, inplace=True)
df['area'] = df['area'].apply(lambda x: float(x.split()[0]))

clean_data = df.copy()

print(clean_data)

clean_data.info()

     house_id        city  sale_price  ... bedrooms     house_type   area
0     1217792  Silvertown       55943  ...        2  Semi-detached  107.8
1     1900913  Silvertown      384677  ...        5       Detached  498.8
2     1174927   Riverford      281707  ...        6       Detached  542.5
3     1773666  Silvertown      373251  ...        6       Detached  528.4
4     1258487  Silvertown      328885  ...        5       Detached  477.1
...       ...         ...         ...  ...      ...            ...    ...
1495  1123892   Riverford      198661  ...        5       Detached  432.2
1496  1327295   Poppleton      358304  ...        6       Detached  599.8
1497  1058161   Riverford      176612  ...        4       Detached  359.1
1498  1822037    Teasdale      197827  ...        3       Detached  253.7
1499  1679272   Poppleton      162099  ...        3       Detached  268.7

[1500 rows x 8 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns 

# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [76]:
# Group the data by number of bedrooms and calculate average sale price and variance
price_by_rooms = clean_data.groupby('bedrooms')['sale_price'].agg(['mean', 'var']).reset_index()

# Rename columns
price_by_rooms.rename(columns={'mean': 'avg_price', 'var': 'var_price'}, inplace=True)

# Round values to 1 decimal place
price_by_rooms['avg_price'] = price_by_rooms['avg_price'].round(1)
price_by_rooms['var_price'] = price_by_rooms['var_price'].round(1)

print(price_by_rooms)

   bedrooms  avg_price     var_price
0         2    67076.4  5.652896e+08
1         3   154665.1  2.378289e+09
2         4   234704.6  1.725211e+09
3         5   301515.9  2.484328e+09
4         6   375741.3  3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [77]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

train_data = pd.read_csv('train.csv')
validation_data = pd.read_csv('validation.csv')

# Convert Detached to 3, Semi-detached to 2, and Terraced to 1
train_data['house_type'] = train_data['house_type'].map({'Terraced': 1, 'Semi-detached': 2, 'Detached': 3})

validation_data['house_type'] = validation_data['house_type'].map({'Terraced': 1, 'Semi-detached': 2, 'Detached': 3})

# Convert Teasdale to 4, Silvertown to 3, Poppleton to 2, and Riverford to 1
train_data['city'] = train_data['city'].map({'Riverford': 1, 'Poppleton': 2, 'Silvertown': 3, 'Teasdale': 4})

validation_data['city'] = validation_data['city'].map({'Riverford': 1, 'Poppleton': 2, 'Silvertown': 3, 'Teasdale': 4})

# 1.
## Features and target
features = train_data[['city', 'months_listed', 'bedrooms', 'house_type', 'area']]
target = train_data['sale_price']

## Modeling
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

## Evaluate the model
print(model.intercept_, model.coef_, model.score(features, target), '\n')
print('y = -188614.73 + 32883.69*X1 + 638.67*X2 + 3788.66*X3 + 32808.15*X4 + 622.59*X5 \n')
print('R^2 = 96.61% \n')

# 2.
validation_features = validation_data[['city', 'months_listed', 'bedrooms', 'house_type', 'area']]

predicted_prices = model.predict(validation_features)

base_result = pd.DataFrame({
    'house_id': validation_data['house_id'],
    'price': predicted_prices}
)

base_result.head()

# RMSE
mse = mean_squared_error(y_test, y_pred)

rmse = (mse ** 0.5)
print('Root Mean Squared Error (Linear Regression):', rmse)

-188614.72893329395 [32883.6898417    638.66808508  3788.65821275 32808.14735775
   622.59129901] 0.9661047751864197 

y = -188614.73 + 32883.69*X1 + 638.67*X2 + 3788.66*X3 + 32808.15*X4 + 622.59*X5 

R^2 = 96.61% 

Root Mean Squared Error (Linear Regression): 23451.255845379852


# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [78]:
from sklearn.ensemble import RandomForestRegressor

train_data = pd.read_csv('train.csv')
validation_data = pd.read_csv('validation.csv')

train_data['house_type'] = train_data['house_type'].map({'Terraced': 1, 'Semi-detached': 2, 'Detached': 3})

validation_data['house_type'] = validation_data['house_type'].map({'Terraced': 1, 'Semi-detached': 2, 'Detached': 3})

train_data['city'] = train_data['city'].map({'Riverford': 1, 'Poppleton': 2, 'Silvertown': 3, 'Teasdale': 4})

validation_data['city'] = validation_data['city'].map({'Riverford': 1, 'Poppleton': 2, 'Silvertown': 3, 'Teasdale': 4})

# 1.
features = train_data[['city', 'months_listed', 'bedrooms', 'house_type', 'area']]
target = train_data['sale_price']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)

print('Feature Importances ->', rf_model.feature_importances_, ':')
print('-The importance of "city" is approximately 7.57%')
print('-The importance of "months_listed" is approximately 0.53%')
print('-The importance of "bedrooms" is approximately 0.08%%')
print('-The importance of "house_type" is approximately 8.77%')
print('-The importance of "area" is approximately 83.05% \n')

# 2.
validation_features = validation_data[['city', 'months_listed', 'bedrooms', 'house_type', 'area']]

predicted_prices_rf = rf_model.predict(validation_features)

compare_result = pd.DataFrame({
    'house_id': validation_data['house_id'],
    'price': predicted_prices_rf}
)

compare_result.head()

# RMSE
mse_rf = mean_squared_error(y_test, y_pred_rf)

rmse_rf = (mse_rf ** 0.5)
print('Root Mean Squared Error (Random Forest):', rmse_rf)

Feature Importances -> [0.07565034 0.00529857 0.00084932 0.08773421 0.83046756] :
-The importance of "city" is approximately 7.57%
-The importance of "months_listed" is approximately 0.53%
-The importance of "bedrooms" is approximately 0.08%%
-The importance of "house_type" is approximately 8.77%
-The importance of "area" is approximately 83.05% 

Root Mean Squared Error (Random Forest): 15393.574772259843
